In [1]:
print("test")

test


In [209]:
import numpy as np
import base64

from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import subprocess
from Bio.SeqFeature import SeqFeature, FeatureLocation
from tempfile import NamedTemporaryFile
import pandas as pd
import streamlit as st

def BLAST(seq,wordsize=12, db='nr_db', BLASTtype="n", flags = 'qstart qend sseqid sframe pident slen sseq length sstart send qlen'):
    query = NamedTemporaryFile()
    tmp = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq), id="temp"), query.name, "fasta")

#     subprocess.call(
#         (f'blast{BLASTtype} -query {query.name} -out {tmp.name} ' #pi needed?
#         f'-db {db} -max_target_seqs 20000 -culling_limit 10 -word_size {str(wordsize)} -outfmt "6 {flags}"'),
#         shell=True)
    subprocess.call(
        (f'blast{BLASTtype} -task blastn-short -query {query.name} -out {tmp.name} -perc_identity 95 ' #pi needed?
        f'-db {db} -max_target_seqs 20000 -culling_limit 10 -word_size {str(wordsize)} -outfmt "6 {flags}"'),
        shell=True)
    
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()

    tmp.close()
    query.close()

    inDf=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    inDf=inDf.apply(pd.to_numeric, errors='ignore')

    return inDf
def clean_and_calculate(inDf):

    inDf['qstart'] = inDf['qstart'].astype('int')
    inDf['qstart'] = inDf['qstart']-1
    inDf['qend']   = inDf['qend']-1
    inDf['percmatch'] = (inDf['length']/inDf['slen']*100)
    inDf[['sseqid','type']] = inDf['sseqid'].str.split("|", n=1, expand=True)
    inDf['sseqid'] = inDf['sseqid'].str.replace(".gb","")#gb artifact from BLASTdb
    inDf['abs percmatch'] = 100-abs(100-inDf['percmatch'])#eg changes 102.1->97.9
    inDf['pi_permatch'] = (inDf["pident"]*inDf["abs percmatch"])/100
    inDf['score']    = (inDf['pi_permatch']/100)*inDf["length"]
    inDf['qlen']     = (inDf['qlen']/2).astype('int')
    inDf['fragment'] = inDf["percmatch"] < 95
    inDf['Feature']  = inDf['sseqid'].str.replace("_"," ") #idk where this happened in other scripts
    
    wiggleSize = 0.1 #this is the percent "trimmed" on either end eg 0.1 == 80%
    inDf['wiggle'] = (inDf['length'] * wiggleSize).astype(int)
    inDf['wstart'] =  inDf['qstart'] + inDf['wiggle']
    inDf['wend']   =  inDf['qend']   - inDf['wiggle']
    
    inDf=inDf.sort_values(by=["score","length","percmatch"], ascending=[False, False, False])
    
    #subtracts a full plasLen if longer than tot length
    inDf['qstart'] = np.where(inDf['qstart'] >= inDf['qlen'], inDf['qstart'] - inDf['qlen'], inDf['qstart'])    
    inDf['qend']   = np.where(inDf['qend']   >= inDf['qlen'], inDf['qend']   - inDf['qlen'], inDf['qend'])

    inDf['wstart'] = np.where(inDf['wstart'] >= inDf['qlen'], inDf['wstart'] - inDf['qlen'], inDf['wstart'])    
    inDf['wend']   = np.where(inDf['wend']   >= inDf['qlen'], inDf['wend']   - inDf['qlen'], inDf['wend'])

    inDf=inDf.drop_duplicates()
    inDf=inDf.reset_index(drop=True)

    #create a conceptual sequence space
    seqSpace=[]
    for i in inDf.index:
        end    = inDf['qlen'][0]
        wstart = inDf.loc[i]['wstart'] #changed from qstart
        wend   = inDf.loc[i]['wend']   #changed from qend

        sseqid = [inDf.loc[i]['sseqid']]

        if wend < wstart: # if hit crosses ori
            left   = (wend + 1)          * [1]
            center = (wstart - wend - 1) * [0]
            right  = (end  - wstart + 0) * [1]
        else: # if normal
            left   =  wstart             * [0]
            center = (wend - wstart + 1) * [1]
            right  = (end  - wend   - 1) * [0]

        seqSpace.append(sseqid+left+center+right)
    seqSpace=pd.DataFrame(seqSpace,columns=['sseqid']+list(range(0,end)))
    seqSpace=seqSpace.set_index([seqSpace.index,'sseqid'])
    seqSpace.to_csv("./hits.csv")

    ######
    #return seqSpace
    #####

    #filter through overlaps in sequence space
    #creates a list of "droppable" indices
    toDrop=set()
    for i in range(len(seqSpace)):

        qstart = inDf.loc[seqSpace.iloc[i].name[0]]['qstart']
        qend   = inDf.loc[seqSpace.iloc[i].name[0]]['qend']

        if seqSpace.iloc[i].name in toDrop:
            continue #need to test speed up on this
    
        #columnSlice = seqSpace.columns[(seqSpace.iloc[i]==1)] #only column names of hit
        if qstart < qend:
            columnSlice = list(range(qstart, qend+1))
        else:
            columnSlice = list(range(0,qend + 1)) + list(range(qstart, end))

        rowSlice    = seqSpace[columnSlice].any(1) #only the rows that are in the columns of hit
        toDrop      = toDrop|set(seqSpace[rowSlice].loc[i+1:].index) #add the indexs below the current to the drop-set

        # if seqSpace.iloc[i].name == (112, 'mTFP1'):
        #     print(i)
        #     print("======")
        #     print(columnSlice)
        #     print("======")
        #     print(rowSlice)
        #     print("======")
        #     print(set(seqSpace[rowSlice].loc[i+1:].index))

    seqSpace=seqSpace.drop(toDrop)

    inDf=inDf.loc[seqSpace.index.get_level_values(0)] #needs shared index labels in pos 0 to work
    inDf=inDf.reset_index(drop=True)
    
    return inDf

def FeatureLocation_smart(r):
    #creates compound locations if needed
    if r.qend>r.qstart:
        return FeatureLocation(r.qstart, r.qend, r.sframe)
    elif r.qstart>r.qend:
        first=FeatureLocation(r.qstart, r.qlen, r.sframe)
        second=FeatureLocation(0, r.qend, r.sframe)
        if r.sframe == 1 or r.sframe == 0:
            return first+second
        elif r.sframe == -1:
            return second+first
def get_gbk(inDf,inSeq):
    #this could be passed a more annotated df
    inDf=inDf.reset_index(drop=True)

    #adds a FeatureLocation object so it can be used in gbk construction
    inDf['feat loc']=inDf.apply(FeatureLocation_smart, axis=1)
    #make a record
    record = SeqRecord(seq=Seq(inSeq),name='pLannotate')
    record.seq.alphabet=generic_dna
    record.annotations["topology"] = "circular"
    for index in inDf.index:
        record.features.append(SeqFeature(
            inDf.loc[index]['feat loc'],
            type=inDf.loc[index]["type"],
            qualifiers={"label": inDf.loc[index]["sseqid"].replace("_"," "),
            "identity":inDf.loc[index]["pident"],
            "match length":inDf.loc[index]["percmatch"],
            "Other:":inDf.loc[index]["type"]}))

    #converts gbk into straight text
    outfileloc=NamedTemporaryFile()
    with open(outfileloc.name, "w") as handle:
        SeqIO.write(record, handle, "genbank")
    with open(outfileloc.name) as handle:
        record=handle.read()
    outfileloc.close()

    return record
def annotate(inSeq):
    database="../BLAST_dbs/full_snapgene_feature_list_w_types_db"

    #I could just create a seq object? this could catch errors though
    fileloc = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(inSeq),name="pLannotate"), fileloc.name, 'fasta')
    record=list(SeqIO.parse(fileloc.name, "fasta"))
    fileloc.close()

    assert len(record)==1,f"FASTA file contains ~multitudes~ --> please submit a single FASTA file."
    record=record[0]

    query=str(record.seq)*2

    blastDf = BLAST(seq=query,wordsize=12, db=database, BLASTtype="n")
    if blastDf.empty: #if no hits are found
        return blastDf
        
    blastDf = clean_and_calculate(blastDf)

    ######
    #return blastDf
    ######

    smallHits=blastDf[blastDf['slen']<25]
    smallHits=smallHits[smallHits["pident"] >= ((smallHits["slen"]-1)/smallHits["slen"])*100] #allows for 1 mismatch
    smallHits=smallHits[smallHits["percmatch"] >= ((smallHits["slen"]-1)/smallHits["slen"])*100]
    smallHits['small']=True

    normHits=blastDf[blastDf['slen']>=25]
    normHits=normHits[normHits['length']>=18]
    normHits=normHits[normHits['pident']>=95]
    normHits['small']=False

    hits=smallHits.append(normHits)
    #hits=hits.set_index(['Feature'])

    return hits


In [210]:
plas=str(list(SeqIO.parse("../fastas/pTN7-pa1-GFP-Kan.fa", "fasta"))[0].seq)
hits=annotate(plas)
hits

,qstart,qend,sseqid,sframe,pident,slen,sseq,length,sstart,send,...,type,abs percmatch,pi_permatch,score,fragment,Feature,wiggle,wstart,wend,small
15,3761,3777,KS_primer,1,100.000,17,CGAGGTCGACGGTATCG,17,1,17,...,primer_bind,100.000000,100.000000,17.000000,False,KS primer,1,3762,3776,True
0,2116,2976,AmpR_(3),-1,99.768,861,TTACCAATGCTTAATCAGTGAGGCACCTATCTCAGCGATCTGTCTA...,861,861,1,...,CDS,100.000000,99.768000,859.002480,False,AmpR (3),86,2202,2890,False
1,5159,402,NeoR_-_KanR_(4),-1,99.623,795,TCAGAAGAACTCGTCAAGAAGGCGATAGAAGGCGATGCGCTGCGAA...,795,795,1,...,CDS,100.000000,99.623000,792.002850,False,NeoR - KanR (4),79,5238,323,False
2,1332,1707,R6K_gamma_ori,1,99.734,389,GCAGTTCAACCTGTTGATAGTACGTACTAAGCTCTCATGTTTCACG...,376,14,389,...,rep_origin,96.658098,96.400987,362.467712,False,R6K gamma ori,37,1369,1670,False
3,3558,3756,Tn7R,-1,100.000,225,TGTGGGCGGACAATAAAGTCTTAAACTGAACAAAATAGATCTAAAC...,199,225,27,...,mobile_element,88.444444,88.444444,176.004444,True,Tn7R,19,3577,3737,False
4,950,1115,Tn7L,-1,100.000,166,AACCAGATAAGTGAAATCTAGTTCCAAACTATTTTGTCATTTTTAA...,166,166,1,...,mobile_element,100.000000,100.000000,166.000000,False,Tn7L,16,966,1099,False
5,1713,2101,ColE1_ori_chunk,1,100.000,932,CTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGA...,389,544,932,...,source,41.738197,41.738197,162.361588,True,ColE1 ori chunk,38,1751,2063,False
6,2977,3081,AmpR_promoter_(2),-1,100.000,105,ACTCTTCCTTTTTCAATATTATTGAAGCATTTATCAGGGTTATTGT...,105,105,1,...,promoter,100.000000,100.000000,105.000000,False,AmpR promoter (2),10,2987,3071,False
7,3805,3899,lambda_t0_terminator_(2),1,100.000,95,GACTCCTGTTGATAGATCCAGTAATGACCTCAGAACTCCATCTGGA...,95,1,95,...,terminator,100.000000,100.000000,95.000000,False,lambda t0 terminator (2),9,3814,3890,False
8,4870,4956,rrnB_T1_terminator_(2),1,100.000,87,CAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTA...,87,1,87,...,terminator,100.000000,100.000000,87.000000,False,rrnB T1 terminator (2),8,4878,4948,False


In [543]:
plas=str(list(SeqIO.parse("./fastas/pTN7-pa1-GFP-Kan.fa", "fasta"))[0].seq)
hits=annotate(plas)
hits=hits.reset_index(drop=True)

seqSpace=[]
for i in hits.index:
    end    = hits['qlen'][0]
    wstart = hits.loc[i]['wstart']
    wend   = hits.loc[i]['wend']
    
    sseqid = [hits.loc[i]['sseqid']]
    
    if wend < wstart: # if hit crosses ori
        left   = (wend + 1)          * [1]
        center = (wstart - wend - 1) * [0]
        right  = (end  - wstart + 0) * [1]
    else: # if normal
        left   =  wstart             * [0]
        center = (wend - wstart + 1) * [1]
        right  = (end  - wend   - 1) * [0]
    
    seqSpace.append(sseqid+left+center+right)
seqSpace=pd.DataFrame(seqSpace,columns=['sseqid']+list(range(0,end)))
seqSpace=seqSpace.set_index([seqSpace.index,'sseqid'])

toDrop=set()
for i in range(len(seqSpace)):
    if seqSpace.iloc[0].name in toDrop:
        continue #adds no speed up, but I prefer it conceptually
    columnSlice=seqSpace.columns[(seqSpace.iloc[i]==1)] #only columns of hit
    rowSlice=seqSpace[columnSlice].any(1) #only the rows that are in the columns of hit
    toDrop=toDrop|set(seqSpace[rowSlice].loc[i+1:].index) #add the indexs below the current to the drop-set
seqSpace=seqSpace.drop(toDrop)
good_hits=hits.loc[seqSpace.index.get_level_values(0)] #needs shared index labels to work
good_hits.reset_index(drop=True)

,qstart,qend,sseqid,sframe,pident,slen,sseq,length,sstart,send,...,percmatch,type,abs percmatch,pi_permatch,score,fragment,Feature,wiggle,wstart,wend
0,2116,2976,AmpR_(3),-1,99.768,861,TTACCAATGCTTAATCAGTGAGGCACCTATCTCAGCGATCTGTCTA...,861,861,1,...,100.000000,CDS,100.000000,99.768000,859.002480,False,AmpR (3),86,2202,2890
1,5159,402,NeoR_-_KanR_(4),-1,99.623,795,TCAGAAGAACTCGTCAAGAAGGCGATAGAAGGCGATGCGCTGCGAA...,795,795,1,...,100.000000,CDS,100.000000,99.623000,792.002850,False,NeoR - KanR (4),79,5238,323
2,1332,1707,R6K_gamma_ori,1,99.734,389,GCAGTTCAACCTGTTGATAGTACGTACTAAGCTCTCATGTTTCACG...,376,14,389,...,96.658098,rep_origin,96.658098,96.400987,362.467712,False,R6K gamma ori,37,1369,1670
3,3558,3756,Tn7R,-1,100.000,225,TGTGGGCGGACAATAAAGTCTTAAACTGAACAAAATAGATCTAAAC...,199,225,27,...,88.444444,mobile_element,88.444444,88.444444,176.004444,True,Tn7R,19,3577,3737
4,950,1115,Tn7L,-1,100.000,166,AACCAGATAAGTGAAATCTAGTTCCAAACTATTTTGTCATTTTTAA...,166,166,1,...,100.000000,mobile_element,100.000000,100.000000,166.000000,False,Tn7L,16,966,1099
5,1713,2101,ColE1_ori_chunk,1,100.000,932,CTGGTAACAGGATTAGCAGAGCGAGGTATGTAGGCGGTGCTACAGA...,389,544,932,...,41.738197,source,41.738197,41.738197,162.361588,True,ColE1 ori chunk,38,1751,2063
6,2977,3081,AmpR_promoter_(2),-1,100.000,105,ACTCTTCCTTTTTCAATATTATTGAAGCATTTATCAGGGTTATTGT...,105,105,1,...,100.000000,promoter,100.000000,100.000000,105.000000,False,AmpR promoter (2),10,2987,3071
7,3805,3899,lambda_t0_terminator_(2),1,100.000,95,GACTCCTGTTGATAGATCCAGTAATGACCTCAGAACTCCATCTGGA...,95,1,95,...,100.000000,terminator,100.000000,100.000000,95.000000,False,lambda t0 terminator (2),9,3814,3890
8,4870,4956,rrnB_T1_terminator_(2),1,100.000,87,CAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTA...,87,1,87,...,100.000000,terminator,100.000000,100.000000,87.000000,False,rrnB T1 terminator (2),8,4878,4948
9,3313,3412,bom,1,100.000,141,CTGGCTTAACTATGCGGCATCAGAGCAGATTGTACTGAGAGTGCAC...,100,42,141,...,70.921986,misc_feature,70.921986,70.921986,70.921986,True,bom,10,3323,3402


In [560]:
plas=str(list(SeqIO.parse("./fastas/pTN7-pa1-GFP-Kan.fa", "fasta"))[0].seq)
hits=annotate(plas)
hits=hits.reset_index(drop=True)

seqSpace=[]
for i in hits.index:
    end    = hits['qlen'][0]
    wstart = hits.loc[i]['wstart']
    wend   = hits.loc[i]['wend']
    
    sseqid = [hits.loc[i]['sseqid']]
    
    if wend < wstart: # if hit crosses ori
        left   = (wend + 1)          * [1]
        center = (wstart - wend - 1) * [0]
        right  = (end  - wstart + 0) * [1]
    else: # if normal
        left   =  wstart             * [0]
        center = (wend - wstart + 1) * [1]
        right  = (end  - wend   - 1) * [0]
    
    seqSpace.append(sseqid+left+center+right)
seqSpace=pd.DataFrame(seqSpace,columns=['sseqid']+list(range(0,end)))
seqSpace=seqSpace.set_index([seqSpace.index,'sseqid'])

seqSpace

,,0,1,2,3,4,5,6,7,8,9,...,5541,5542,5543,5544,5545,5546,5547,5548,5549,5550
,sseqid,,,,,,,,,,,,,,,,,,,,,
0,AmpR_(3),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,AmpR_(4),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,AmpR,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NeoR_-_KanR_(4),1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,bla(M),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,HIV-1_gag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
86,PpHIS4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,HIV-1_pol,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
x=2394
chunk=5
np.floor(x/chunk)

478.0

In [52]:
import numpy as np
from math import pi
from bokeh.models import HoverTool, ColumnDataSource, WheelZoomTool, Range1d
from bokeh.plotting import figure
from bokeh.io import show
import pandas as pd

hover = HoverTool(names=["1"])
plotSize=.35
plotDimen=800
TOOLTIPS='<font size="3"><b>@Feature</b> — @Type   @pi_permatch_int%</font> <br> @Description'


x_range = Range1d(-plotSize, plotSize, bounds=(-.5, .5), min_interval=.1)
y_range = Range1d(-plotSize, plotSize, bounds=(-.5, .5), min_interval=.1)
p = figure(plot_height=plotDimen,plot_width=plotDimen, title="",
            toolbar_location=None,toolbar_sticky=False, match_aspect=True,
            sizing_mode='scale_width', tools=[hover,'pan'], tooltips=TOOLTIPS,
            #x_range=(-plotSize, plotSize), y_range=(-plotSize, plotSize))
            x_range=x_range, y_range=y_range)


#calculate chunk size for drawing lines
plasLen=4800
if plasLen <= 2000:
        chunkRound= plasLen//1000*100
if plasLen <= 1000:
    chunkRound= plasLen//100*10
else:
    chunkRound=500
chunkSize = (plasLen // 5) // chunkRound*chunkRound
chunks = pd.Series(range(0, chunkSize * 5, chunkSize))
chunks = (chunks/plasLen) * 2 * pi

theta=(pi/2)-chunks #rotate for canvas

offset = .2
Lx0 = np.cos(theta) * offset
Ly0 = np.sin(theta) * offset
longRadius = offset * 1.2
Lx1 = np.cos(theta) * longRadius
Ly1 = np.sin(theta) * longRadius

lineX=list(zip(Lx0,Lx1))
lineY=list(zip(Ly0,Ly1))

ticks = ColumnDataSource(pd.DataFrame(list(zip(lineX,lineY)),columns=['lineX','lineY']))
p.multi_line(xs="lineX", ys="lineY", line_color="black", line_width=3,
            level="overlay", line_cap='round', alpha = .5, source = ticks)
show(p)

In [173]:
plasLen=6015
# if plasLen > 3000 and plasLen <= 5000:
#     chunkRound= 1000
# elif plasLen <= 2000:
#     chunkRound= plasLen//1000*100
# elif plasLen <= 1000:
#     chunkRound= plasLen//100*10
# else:
#     chunkRound=500
# chunkSize = (plasLen // 5) // 500*chunkRound
chunkSize = int(np.ceil((plasLen//5)/500)*500)
chunks = pd.Series(range(0, plasLen-int(chunkSize/2), chunkSize))
chunks

0       0
1    1500
2    3000
3    4500
dtype: int64

In [171]:
np.ceil((plasLen//5)/500)

3.0

1250.0

In [157]:
pl=6701
round((pl//5)/500)*500

1500

In [103]:
plasLen=3900
if plasLen <= 2000:
        chunkRound= plasLen//1000*100
if plasLen <= 1000:
    chunkRound= plasLen//100*10
else:
    chunkRound=500

def round_nearest(x,num=500):
    return int(np.floor(float(x)/num)*num)

chunkSize = round_nearest(plasLen // 5)
chunks = pd.Series(range(0, plasLen, chunkSize))
chunks

0       0
1     500
2    1000
3    1500
4    2000
5    2500
6    3000
7    3500
dtype: int64

In [87]:
(plasLen // 5) // 500 * 500

500

1000

blastx -query ./fastas/pBTK562.fa -out ./out.blast -db ./BLAST_dbs/addgene_CDS_20-9-3 -max_target_seqs 20000 -culling_limit 10 -word_size 6 -outfmt "6 qstart qend sseqid sframe pident slen sseq length sstart send qlen"

In [55]:
def dBLAST(seq, db='nr_db', flags = 'bitscore sseqid pident qstart qend sseq sstart send length slen qlen qframe'): 
    query = NamedTemporaryFile()
    SeqIO.write(SeqRecord(Seq(seq*2), id="temp"), query.name, "fasta")
    tmp = NamedTemporaryFile(suffix=".m8")
    subprocess.call(
        'diamond blastx -d /Users/mattmcguffie/Desktop/uniprot/swissprot -q ' + 
        query.name + ' -o ' + tmp.name + ' --taxonmap /Users/mattmcguffie/Documents/GitHub/pLannotate/BLAST_dbs/taxdb.btd --outfmt 6 '+flags, shell=True)
    with open(tmp.name, "r") as file_handle:  #opens BLAST file
        align = file_handle.readlines()
        
    tmp.close() 
    query.close() 
    align=pd.DataFrame([ele.split() for ele in align],columns=flags.split())
    if not align.empty:
        #align['sseqid']=align['sseqid'].str.split("|",expand=True)[1]
        align['pident']=align['pident'].astype(float)
        align['slen']=align['slen'].astype(int)
        align['length']=align['length'].astype(int)
        align['qlen']=align['qlen'].astype(int)
        align['pmatch']=align['length']/align['qlen']
        align['qframe']=align['qframe'].astype(int)
        align['frame']=np.sign(align['qframe'])
        align['fractionmatch']=align['length']/align['slen']
        align['bitscore']=align['bitscore'].astype(float)
        align['score']=align['pident']*align['length'].astype(int)*align['fractionmatch']
        align=align.sort_values(by=['score'],ascending=False)
    return align 

In [56]:
dBLAST(plas)

,bitscore,sseqid,pident,qstart,qend,sseq,sstart,send,length,slen,qlen,qframe,pmatch,frame,fractionmatch,score
0,1345.1,sp|P07112|MOBA2_ECOLX,98.9,10111,7985,MAIYHLTAKTGSRSGGQSARAKADYIQREGKYARDMDEVLHAESGH...,1,709,709,709,16202,-2,0.043760,-1,1.000000,70120.100000
1,1268.4,sp|P07112|MOBA2_ECOLX,98.8,2010,1,MAIYHLTAKTGSRSGGQSARAKADYIQREGKYARDMDEVLHAESGH...,1,670,670,709,16202,-3,0.041353,-1,0.944993,62554.753173
3,556.2,sp|P20356|REPJ_ECOLX,100.0,15572,14736,MATHKPINILEAFAAAPPPLDYVLPNMVAGTVGALVSPGGAGKSML...,1,279,279,279,16202,-1,0.017220,-1,1.000000,27900.000000
4,556.2,sp|P20356|REPJ_ECOLX,100.0,7471,6635,MATHKPINILEAFAAAPPPLDYVLPNMVAGTVGALVSPGGAGKSML...,1,279,279,279,16202,-2,0.017220,-1,1.000000,27900.000000
5,522.3,sp|P0AG05|S3AD_ECOLX,100.0,5414,4626,MREAVIAEVSTQLSEVVGVIERHLEPTLLAVHLYGSAVDGGLKPHS...,1,263,263,263,16202,-1,0.016233,-1,1.000000,26300.000000
6,522.3,sp|P0AG05|S3AD_ECOLX,100.0,13515,12727,MREAVIAEVSTQLSEVVGVIERHLEPTLLAVHLYGSAVDGGLKPHS...,1,263,263,263,16202,-3,0.016233,-1,1.000000,26300.000000
7,516.2,sp|P0AG06|S3AD_SHIFL,99.6,5414,4626,MREAVIAEVSTQLSEVVGVIERHLEPTLLAVHLYGSAVDGGLKPHS...,1,262,263,262,16202,-1,0.016233,-1,1.003817,26294.780153
8,516.2,sp|P0AG06|S3AD_SHIFL,99.6,13515,12727,MREAVIAEVSTQLSEVVGVIERHLEPTLLAVHLYGSAVDGGLKPHS...,1,262,263,262,16202,-3,0.016233,-1,1.003817,26294.780153
9,514.6,sp|P08881|S3AD_KLEPN,99.2,5414,4626,MREAVIAKVSTQLSEVVGVIERHLEPTLLAVHLYGSAVDGGLKPHS...,1,262,263,262,16202,-1,0.016233,-1,1.003817,26189.178626
10,514.6,sp|P08881|S3AD_KLEPN,99.2,13515,12727,MREAVIAKVSTQLSEVVGVIERHLEPTLLAVHLYGSAVDGGLKPHS...,1,262,263,262,16202,-3,0.016233,-1,1.003817,26189.178626
